# 합성 테스트 데이터셋 생성

**왜 합성 테스트 데이터(Synthetic Test Dataset) 인가?**

RAG(검색 증강 생성) 증강 파이프라인의 성능을 평가하는 것은 매우 중요합니다. 

그러나 문서에서 수백 개의 QA(질문-문맥-응답) 샘플을 수동으로 생성하는 것은 시간과 노동력이 많이 소요될 수 있습니다. 또한 사람이 만든 질문은 철저한 평가에 필요한 복잡성 수준에 도달하기 어려워 궁극적으로 평가의 품질에 영향을 미칠 수 있습니다. 

합성 데이터 생성을 사용하면 데이터 집계 프로세스에서 **개발자의 시간을 90%** 까지 줄일 수 있습니다.

- RAGAS: https://docs.ragas.io/en/latest/concepts/testset_generation.html

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


## 문서 전처리

문서를 로드 합니다.

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

19

각 문서 객체에는 `metadata` 를 통해 액세스할 수 있는 문서에 대한 추가 정보를 저장하는 데 사용할 수 있는 메타데이터 사전이 포함되어 있습니다. 

메타데이터 사전에는 `filename` 이라는 키가 포함되어 있는지 확인하세요. 

이 키는 Test datasets 생성 프로세스에서 활용될 것이므로. 메타데이터의 `filename` 속성은 동일한 문서에 속한 청크를 식별하는 데 사용됩니다. 

In [4]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

## 데이터셋 생성

In [5]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o-mini")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o-mini")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

DocumentStore를 초기화합니다. 사용자 정의 LLM과 임베딩을 사용합니다.

In [6]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

TestSet 을 생성합니다.

In [7]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

**질문의 유형별 분포**

- simple: 간단한 질문
- reasoning: 추론이 필요한 질문
- multi_context: 여러 맥락을 고려해야 하는 질문
- conditional: 조건부 질문

In [9]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}

- documents: 문서 데이터
- test_size: 생성할 질문의 수
- distributions: 질문 유형별 분포
- with_debugging_logs: 디버깅 로그 출력 여부

In [ ]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs, test_size=10, distributions=distributions, with_debugging_logs=True
)

In [12]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the key considerations regarding AI f...,[∙ 3개국은 기반모델 전반에 대한 규제가 기술 중립적이고 위험 기반의 AI 규제 ...,Key considerations regarding AI foundation mod...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What are the key features and improvements of ...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라우...,The key features and improvements of the LLM '...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What is the purpose of the AAAI Conference?,[Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CT...,The purpose of the AAAI Conference is to advan...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What role does content moderation play in the ...,"[£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\nn...",Content moderation plays a crucial role in AI ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What FTC concerns exist on AI comp. & consumer...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 연방거래위...,The FTC has raised concerns regarding AI-relat...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
5,"What key algorithms, architectures, and functi...","[구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY C...",The context does not provide specific details ...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
6,What strategies ensure safe and ethical AI adv...,[문제를 방지하는 조치를 확대\n∙ 형사사법 시스템에서 AI 사용 모범사례를 개발하...,The context discusses various strategies for e...,multi_context,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
7,What key metrics gauge AI app effectiveness an...,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n빌 게이츠, A...",The key metrics that gauge AI app effectivenes...,multi_context,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
8,What does the complementarity principle mean f...,"[영국 옥스퍼드 인터넷 연구소, AI 기술자의 임금이 평균 21% 높아\nKEY C...",The complementarity principle in AI suggests t...,conditional,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
9,How might new copyright laws affect FTC's AI r...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 연방거래위...,The context does not provide specific informat...,conditional,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


DataFrame 에 저장된 데이터셋을 csv 파일로 저장합니다

In [13]:
# DataFrame의 상위 5개 행 출력
test_df.head()

# DataFrame을 CSV 파일로 저장
test_df.to_csv("data/ragas_synthetic_dataset.csv", index=False)